# Business Market Analysis
___

## Introduction / Business Problem

This analysis is meant to help a business venture choose between locations. Specifically we will look at Mexican retaraunts in Los Angeles. Finding a neighborhood that is not saturated with other Mexican restaunts would be a great place to start!

## Data

In [ ]:
import numpy as np
import pandas as pd
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
import urllib.request
from urllib.request import urlopen
import time
from bs4 import BeautifulSoup
!pip install pgeocode
import pgeocode

<div class="alert alert-block alert-danger">
<b>Delete this cell before publishing
</div>



In [ ]:
# Foursquare API authentication
CLIENT_ID = '' 
CLIENT_SECRET = '' 
ACCESS_TOKEN = '' 
VERSION = '20180604'
LIMIT = 30

### scrape for list of neighborhoods in LA


First we need to set some variables, including defining the source of our information. We will be scraping LAalminac.com to get out communities and zip codes.

In [ ]:
url = "http://www.laalmanac.com/communications/cm02_communities.php" 
data = requests.get(url).text 
soup = BeautifulSoup(data,"html.parser")

Here we are creating a list, finding our cells, cleaning some unnececary characters, and appending our list.

In [ ]:
import re

table=soup.find('table')
list_rows = []
for row in table:
    cells = soup.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

[Acton, 93510, Agoura Hills, 91301, Agoura Hills (PO Boxes), 91376, Agua Dulce, 91390, Alhambra, 91801, 91803, Alhambra (Non-Geographic Zip Code Within 91801), 91804, Alhambra (PO Boxes), 91802, 91896, 91899, Altadena, 91001, Altadena (PO Boxes), 91003, Americana at Brand &amp; Glendale Galleria (Glendale), 91210, Arcadia, 91006, 91007, Arcadia (PO Boxes), 91066, 91077, Arleta (Los Angeles), 91331, Arlington Heights (Los Angeles), 90019, Artesia, 90701, Artesia (PO Boxes), 90702, Athens, 90044, Atwater Village (Los Angeles), 90039, Avalon (PO Boxes), 90704, Avocado Heights, 91746, Azusa, 91010, 91702, Baldwin Hills (Los Angeles), 90008, Baldwin Park, 91706, Bassett, 91746, Bel Air Estates (Los Angeles), 90049, Bel Air Estates, Beverly Glen (Los Angeles), 90077, Bell, 90201, 90270, Bell (PO Boxes), 90202, Bell (Shared Firms), 90096, Bell Canyon, 91307, Bell Gardens, 90201, Bell Gardens (PO Boxes), 90202, Bellflower, 90706, Bellflower (PO Boxes), 90707, Belmont Shore (Long Beach), 90803,

str

Next we must split the data at the commas.

In [ ]:
s = clean2.split(',')
s

['[Acton',
 ' 93510',
 ' Agoura Hills',
 ' 91301',
 ' Agoura Hills (PO Boxes)',
 ' 91376',
 ' Agua Dulce',
 ' 91390',
 ' Alhambra',
 ' 91801',
 ' 91803',
 ' Alhambra (Non-Geographic Zip Code Within 91801)',
 ' 91804',
 ' Alhambra (PO Boxes)',
 ' 91802',
 ' 91896',
 ' 91899',
 ' Altadena',
 ' 91001',
 ' Altadena (PO Boxes)',
 ' 91003',
 ' Americana at Brand &amp; Glendale Galleria (Glendale)',
 ' 91210',
 ' Arcadia',
 ' 91006',
 ' 91007',
 ' Arcadia (PO Boxes)',
 ' 91066',
 ' 91077',
 ' Arleta (Los Angeles)',
 ' 91331',
 ' Arlington Heights (Los Angeles)',
 ' 90019',
 ' Artesia',
 ' 90701',
 ' Artesia (PO Boxes)',
 ' 90702',
 ' Athens',
 ' 90044',
 ' Atwater Village (Los Angeles)',
 ' 90039',
 ' Avalon (PO Boxes)',
 ' 90704',
 ' Avocado Heights',
 ' 91746',
 ' Azusa',
 ' 91010',
 ' 91702',
 ' Baldwin Hills (Los Angeles)',
 ' 90008',
 ' Baldwin Park',
 ' 91706',
 ' Bassett',
 ' 91746',
 ' Bel Air Estates (Los Angeles)',
 ' 90049',
 ' Bel Air Estates',
 ' Beverly Glen (Los Angeles)',
 ' 9

Here we have the bulk of our data cleaning! See the comments for more information on what is happening but the basic idea is to take a list of strings and turn some of the items into int, while also getting rid of some extra zip codes we wont be using and fix a couple formatting problems. We will also be grabbing some latitude and longitude data while we are at it

In [ ]:
#remove spaces
s1 = [s.strip(' ') for s in s]
#create empty list
a = []
nomi = pgeocode.Nominatim('US')
#create variable to use for itteration
cityNumber = 0
#create boolean variables for determining if last itterated was a string or int
lastWasZIP = False
lastwasletter = False
#for loop to convert zip codes to int and group city with zip code
for x in s1:
  if not x[1].isdigit() and lastwasletter == False: #if current item is NOT a number AND last item WAS
    if cityNumber == 0:                                 #if city number is 0->
      a.append([x[1:],10000,0,0])                           #append with 10000 for easy to spot error catching and remove 1st "["
      cityNumber = cityNumber + 1                       #move to next variable
      lastwasletter = True                              #last item WAS a letter
    else:
      if not str(a[cityNumber-1][0][0:5]) in x:         #if not a str that matches previous cities 1st 6 characters
        a.append([x,10000,0,0])                             #append with 10000 for easy to spot error catching
        cityNumber = cityNumber + 1                     #move to next variable
        lastWasZIP = False                              #last item was NOT a number
        lastwasletter = True                            #last item WAS a letter
  elif lastWasZIP == False and x[1].isdigit():      #if last item was NOT a number AND current IS
      if x[len(x)-1] != "]":                            
        a[cityNumber-1][1] = int(x)                     #replace 10000 with int version or this line of x     
        a[cityNumber-1][2] = nomi.query_postal_code(int(x)).latitude
        a[cityNumber-1][3] = nomi.query_postal_code(int(x)).longitude
        lastWasZIP = True
        lastwasletter = False
      else:
        a[cityNumber-1][1] = int(x[:-1])                #replace 10000 with int version without last character of x
        a[cityNumber-1][2] = nomi.query_postal_code(int(x[:-1])).latitude
        a[cityNumber-1][3] = nomi.query_postal_code(int(x[:-1])).longitude
        lastWasZIP = True
        lastwasletter = False

In [ ]:
a

[['Acton', 93510, 34.4835, -118.1959],
 ['Agoura Hills', 91301, 34.1227, -118.7573],
 ['Agua Dulce', 91390, 34.4684, -118.5261],
 ['Alhambra', 91801, 34.0914, -118.1293],
 ['Altadena', 91001, 34.1912, -118.1392],
 ['Americana at Brand &amp; Glendale Galleria (Glendale)',
  91210,
  34.1425,
  -118.2551],
 ['Arcadia', 91006, 34.1324, -118.0264],
 ['Arleta (Los Angeles)', 91331, 34.2556, -118.4208],
 ['Arlington Heights (Los Angeles)', 90019, 34.0482, -118.3343],
 ['Artesia', 90701, 33.8654, -118.0731],
 ['Athens', 90044, 33.9551, -118.2901],
 ['Atwater Village (Los Angeles)', 90039, 34.1121, -118.2594],
 ['Avalon (PO Boxes)', 90704, 33.332, -118.3437],
 ['Avocado Heights', 91746, 34.0443, -117.9862],
 ['Azusa', 91010, 34.1407, -117.9567],
 ['Baldwin Hills (Los Angeles)', 90008, 34.0116, -118.3411],
 ['Bassett', 91746, 34.0443, -117.9862],
 ['Bel Air Estates (Los Angeles)', 90049, 34.066, -118.47399999999999],
 ['Beverly Glen (Los Angeles)', 90077, 34.1112, -118.4502],
 ['Bell', 90201, 3

Lets make it a data frame and have a look!

In [ ]:
df = pd.DataFrame(a, columns =['Neighborhood', 'PostalCode', 'Latitude', 'Longitude'])
df

,Neighborhood,PostalCode,Latitude,Longitude
0,Acton,93510,34.4835,-118.1959
1,Agoura Hills,91301,34.1227,-118.7573
2,Agua Dulce,91390,34.4684,-118.5261
3,Alhambra,91801,34.0914,-118.1293
4,Altadena,91001,34.1912,-118.1392
...,...,...,...,...
256,Wilshire Center (Los Angeles),90004,34.0762,-118.3029
257,Wilsona Gardens,93535,34.7131,-117.8783
258,Windsor Hills (Los Angeles),90043,33.9871,-118.3321
259,Winnetka (Los Angeles),91306,34.2092,-118.5749


## define a function to locate nearby venues
___


This will grab all the restaurants labeled "Mexican"

In [32]:
def getNearbyVenues(Neighborhood, PostalCode, Latitude, Longitude, radius=500):
    
    venues_list=[]
    for name, postalcode, lat, lng in zip(Neighborhood, PostalCode, Latitude, Longitude):
        if postalcode != 91797:
          #print(name)
            
          # create the API request URL
          url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
              CLIENT_ID, 
              CLIENT_SECRET, 
              VERSION, 
              lat, 
              lng, 
              radius, 
              LIMIT)
            
          # make the GET request
          #print(requests.get(url).json())
          results = requests.get(url).json()["response"]['groups'][0]['items']
          #print(results)
          #print("\n")
          # return only relevant information for each nearby venue
          for v in results:
            if v['venue']['categories'][0]['shortName'] == 'Mexican':
          #print("Here1\n")
              venues_list.append([(
               name, 
               lat, 
               lng, 
               v['venue']['id'],
               v['venue']['name'], 
               v['venue']['location']['lat'], 
               v['venue']['location']['lng'],  
               v['venue']['categories'][0]['name'])])
    #print("Here2\n")
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Lat',
                  'Neighborhood Long',
                  'ID',
                  'Venue', 
                  'Venue Lat',
                  'Venue Long',
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
LA_venues = getNearbyVenues(Neighborhood=df['Neighborhood'],PostalCode=df['PostalCode'], Latitude=df['Latitude'],Longitude=df['Longitude'])

In [34]:
LA_venues

,Neighborhood,Neighborhood Lat,Neighborhood Long,ID,Venue,Venue Lat,Venue Long,Venue Category
0,Arcadia,34.1324,-118.0264,4b42664ef964a52023d325e3,Taco Lita,34.130003,-118.027064,Mexican Restaurant
1,Arlington Heights (Los Angeles),34.0482,-118.3343,524b20ba11d298598ce1c917,Chipotle Mexican Grill,34.048183,-118.336012,Mexican Restaurant
2,Arlington Heights (Los Angeles),34.0482,-118.3343,4aeb7b84f964a52088c221e3,El Compita,34.048592,-118.332846,Mexican Restaurant
3,Azusa,34.1407,-117.9567,4bca374f511f95219e57afc7,Tacos Ensenada,34.140143,-117.957711,Mexican Restaurant
4,Baldwin Hills (Los Angeles),34.0116,-118.3411,50f6caa4b0b9aa3a48bead1d,Chipotle Mexican Grill,34.013284,-118.336625,Mexican Restaurant
...,...,...,...,...,...,...,...,...
174,Willowbrook,33.9293,-118.2463,4d61d133e4fe54815b25909e,Tacos El Bronco,33.929620,-118.246058,Mexican Restaurant
175,Wilmington (Los Angeles),33.7855,-118.2645,4bb64c126edc76b04cb6301c,Mariscos La Paz,33.787514,-118.263280,Mexican Restaurant
176,Wilshire Center (Los Angeles),34.0762,-118.3029,49e8c872f964a5207c651fe3,Cactus Mexican Food,34.076194,-118.304147,Mexican Restaurant
177,Wilshire Center (Los Angeles),34.0762,-118.3029,4c2932e49eb1952105ad2959,Casa Carnitas,34.076540,-118.297723,Mexican Restaurant


## map venues in relation to neighborhoods

Here is an interactive map that shows all the venues in blue and all the neighborhoods in red.

In [35]:
map_LA = folium.Map(location=[34.1324,-118.0264],zoom_start=10)

for ven_lat,ven_lng,neighborhood, venue in zip(LA_venues['Venue Lat'],LA_venues['Venue Long'],LA_venues['Neighborhood'],LA_venues['Venue']):
    label = '{}, {}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [ven_lat,ven_lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_LA)

for n_lat,n_lng,neighborhood in zip(LA_venues['Neighborhood Lat'],LA_venues['Neighborhood Long'],LA_venues['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [n_lat,n_lng],
    radius=3,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.4,
    parse_html=False).add_to(map_LA)

map_LA

In [36]:
count = (LA_venues.groupby(['Neighborhood']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='count')) 
count

,Neighborhood,count
0,San Fernando,6
1,Sylmar (Los Angeles),6
2,Mission Hills (Los Angeles),6
3,Maywood,5
4,Boyle Heights (Los Angeles),4
...,...,...
99,Leimert Park (Los Angeles),1
100,Lake Balboa (Los Angeles),1
101,La Habra Heights,1
102,Koreatown (Los Angeles),1


Wile I dont want to pick an area that is saturated I also don't want to pick an area with no demand for a Mexican Restaruant. Since I like the neighborhood University Park and it has enough venues to show demand lets look a little deeper!

In [37]:
count.loc[count['Neighborhood'] == 'University Park (Los Angeles)']

,Neighborhood,count
7,University Park (Los Angeles),3


In [40]:
UP_venues = LA_venues.loc[LA_venues['Neighborhood'] == 'University Park (Los Angeles)']
UP_venues.reset_index
UP_venues

,Neighborhood,Neighborhood Lat,Neighborhood Long,ID,Venue,Venue Lat,Venue Long,Venue Category
153,University Park (Los Angeles),34.0294,-118.2871,4fac7382e4b021d7498d6385,La Valentina,34.029765,-118.291645,Mexican Restaurant
154,University Park (Los Angeles),34.0294,-118.2871,5179acca498e4dd185fd3b87,Monarca,34.030336,-118.291802,Mexican Restaurant
155,University Park (Los Angeles),34.0294,-118.2871,4bfddcccf61dc9b6e9bc9fde,Cilantro Mexican Grill,34.025729,-118.285055,Mexican Restaurant


## La Valentina

In [42]:
venue_id = UP_venues.iloc[0][3]
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

In [43]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$$', 'priceTier': 2}],
    'name': 'Price',
    'summary': '$$',
    'type': 'price'},
   {'count': 3,
    'items': [{'displayName': 'Reservations', 'displayValue': 'No'}],
    'name': 'Reservations',
    'type': 'reservations'},
   {'count': 7,
    'items': [{'displayName': 'Credit Cards',
      'displayValue': 'Yes (incl. Discover & Visa)'}],
    'name': 'Credit Cards',
    'summary': 'Credit Cards',
    'type': 'payments'},
   {'count': 1,
    'items': [{'displayName': 'Outdoor Seating', 'displayValue': 'Yes'}],
    'name': 'Outdoor Seating',
    'summary': 'Outdoor Seating',
    'type': 'outdoorSeating'}]},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'bestPhoto': {'createdAt': 1379443191,
  'height': 717,
  'id': '5238a1f7498e93328a0d057a',
  'prefix': 'https://fastly.4sqi.net/img/general/',
  'source': {

In [57]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.0


In [53]:
try:
    print(result['response']['venue']['likes']['summary'])
except:
    print('This venue has not been rated yet.')

14 Likes


## Monarca

In [62]:
venue_id1 = UP_venues.iloc[1][3]
url1 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

In [63]:
venue_id1

'5179acca498e4dd185fd3b87'

In [64]:
result1 = requests.get(url1).json()
print(result1['response']['venue'].keys())
result1['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$', 'priceTier': 1}],
    'name': 'Price',
    'summary': '$',
    'type': 'price'},
   {'count': 3,
    'items': [{'displayName': 'Reservations', 'displayValue': 'No'}],
    'name': 'Reservations',
    'type': 'reservations'},
   {'count': 7,
    'items': [{'displayName': 'Credit Cards', 'displayValue': 'No'}],
    'name': 'Credit Cards',
    'summary': 'No Credit Cards',
    'type': 'payments'}]},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'bestPhoto': {'createdAt': 1484376519,
  'height': 1440,
  'id': '5879c9c77b88a705eb7a97bf',
  'prefix': 'https://fastly.4sqi.net/img/general/',
  'source': {'name': 'Swarm for iOS', 'url': 'https://www.swarmapp.com'},
  'suffix': '/2705098_J9kyd0opR05-Nt2wySC2cukfZNocqosrCzXnLp9suZE.jpg',
  'visibility': 'public',
  'width': 1440},
 'canonicalUrl': 'https://foursquare.c

In [65]:
try:
    print(result1['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [76]:
try:
    print(result1['response']['venue']['likes'])
except:
    print('This venue has not been rated yet.')

{'count': 1, 'groups': [{'type': 'others', 'count': 1, 'items': [{'id': '391656528', 'firstName': 'Reynaldo', 'lastName': 'Sosa', 'gender': 'male', 'countryCode': 'US', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/391656528_1A5vi2C1_MB48N510CnIhJRgBImlN6ci5qQto6pFO6ParF1rdNjzO9EI8ztW1jw6XGIiOn4cY.jpg'}}]}], 'summary': 'Reynaldo Sosa'}


This venue does not have a rating and only 1 like

## Cilantro Mexican Grill

In [67]:
venue_id2 = UP_venues.iloc[2][3]
url2 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

In [69]:
venue_id2

'4bfddcccf61dc9b6e9bc9fde'

In [70]:
result2 = requests.get(url2).json()
print(result2['response']['venue'].keys())
result2['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$', 'priceTier': 1}],
    'name': 'Price',
    'summary': '$',
    'type': 'price'},
   {'count': 3,
    'items': [{'displayName': 'Reservations', 'displayValue': 'No'}],
    'name': 'Reservations',
    'type': 'reservations'},
   {'count': 7,
    'items': [{'displayName': 'Credit Cards', 'displayValue': 'No'}],
    'name': 'Credit Cards',
    'summary': 'No Credit Cards',
    'type': 'payments'}]},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'bestPhoto': {'createdAt': 1484376519,
  'height': 1440,
  'id': '5879c9c77b88a705eb7a97bf',
  'prefix': 'https://fastly.4sqi.net/img/general/',
  'source': {'name': 'Swarm for iOS', 'url': 'https://www.swarmapp.com'},
  'suffix': '/2705098_J9kyd0opR05-Nt2wySC2cukfZNocqosrCzXnLp9suZE.jpg',
  'visibility': 'public',
  'width': 1440},
 'canonicalUrl': 'https://foursquare.c

In [73]:
try:
    print(result2['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [75]:
try:
    print(result2['response']['venue']['likes'])
except:
    print('This venue has not been rated yet.')

{'count': 1, 'groups': [{'type': 'others', 'count': 1, 'items': [{'id': '391656528', 'firstName': 'Reynaldo', 'lastName': 'Sosa', 'gender': 'male', 'countryCode': 'US', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/391656528_1A5vi2C1_MB48N510CnIhJRgBImlN6ci5qQto6pFO6ParF1rdNjzO9EI8ztW1jw6XGIiOn4cY.jpg'}}]}], 'summary': 'Reynaldo Sosa'}


This venue does not have a rating and only 1 like. (From the same person who liked Monarca!)

# Conclusion

It looks like University park may be a great place to start a Mexican Restaraunt! There are only 3 in the neighborhood none of them appear to have good ratings. This could indicate that there is a demand. People seem to want this type of food to keep 3 venues open. There also seems to be opportunity (since the ratings and number of likes are low) to fill that demand with something that people will enjoy. Further market research is still needed but this area looks promising!